## 연령별 count 세어주기
- 'fakefriends.csv' 활용
- age가 13세 이상 19세 미만인 사람들 조회
- 연령 별 count를 DataFrame API 사용해서 구해보기
- ``쿼리쓰지말고`` 

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("exam").getOrCreate()

In [3]:
pwd

'c:\\Users\\apfhd\\SparkExam'

###  DataFrame 활용을 위해 pandas 활용하였으나 이것은 DataFrame API 활용 방안이 아니다.

In [4]:
# DataFrame 활용을 위해 pandas 활용하였으나 이것은 DataFrame API 활용 방안이 아니다.
import pandas as pd
directory="c://Users//apfhd//SparkExam//data//fakefriends.csv"
data =pd.read_csv(f'file:///{directory}')
data.head()

,0,Will,33,385
0,1,Jean-Luc,26,2
1,2,Hugh,55,221
2,3,Deanna,40,465
3,4,Quark,68,21
4,5,Weyoun,59,318


In [5]:
friend_schema=['number','name','age','friend_amount']

In [6]:
df=spark.createDataFrame(data=data,schema=friend_schema)
df.show(5)

+------+--------+---+-------------+
|number|    name|age|friend_amount|
+------+--------+---+-------------+
|     1|Jean-Luc| 26|            2|
|     2|    Hugh| 55|          221|
|     3|  Deanna| 40|          465|
|     4|   Quark| 68|           21|
|     5|  Weyoun| 59|          318|
+------+--------+---+-------------+
only showing top 5 rows



In [7]:
df.createOrReplaceTempView('friends')

In [8]:
query="""
select f.name,f.friend_amount
from friends f
where f.age >= 13 and f.age < 19
"""
spark.sql(query).show()

+-------+-------------+
|   name|friend_amount|
+-------+-------------+
|Beverly|          499|
|  Dukat|          397|
|   Data|          326|
|Beverly|          418|
| Kasidy|           24|
|   Data|          417|
|  Keiko|          472|
| Kasidy|          194|
+-------+-------------+



### DataFrame API 활용하는 모범 답안

In [10]:
# DataFrame API 활용하는 모범 답안
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, FloatType, IntegerType

friend_schema= StructType([
    StructField("id", IntegerType(),True),
    StructField("name",StringType(),True),
    StructField("age",IntegerType(),True),
    StructField("number_friends",IntegerType(),True)
])
datas=spark.read.csv(f'file:///{directory}',schema=friend_schema)

In [11]:
# 정답
friends=datas.filter((datas.age>=13)&(datas.age<=19))
friends.show()

+---+-------+---+--------------+
| id|   name|age|number_friends|
+---+-------+---+--------------+
| 21|  Miles| 19|           268|
| 52|Beverly| 19|           269|
| 54|  Brunt| 19|             5|
|106|Beverly| 18|           499|
|115|  Dukat| 18|           397|
|133|  Quark| 19|           265|
|136|   Will| 19|           335|
|225|   Elim| 19|           106|
|304|   Will| 19|           404|
|341|   Data| 18|           326|
|366|  Keiko| 19|           119|
|373|  Quark| 19|           272|
|377|Beverly| 18|           418|
|404| Kasidy| 18|            24|
|409|    Nog| 19|           267|
|439|   Data| 18|           417|
|444|  Keiko| 18|           472|
|492|  Dukat| 19|            36|
|494| Kasidy| 18|           194|
+---+-------+---+--------------+



### 피드백 : DataFrame API 활용 시 ``스키마를 수동으로 각각 타입을 지정``해주고 query가 아닌 ``filter로 활용``해야한다.